In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
df_train = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
df_test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")
df_submit = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/sample_submission.csv")

In [4]:
df_train.head

<bound method NDFrame.head of                      id        anchor                  target context  score
0      37d61fd2272659b1     abatement  abatement of pollution     A47   0.50
1      7b9652b17b68b7a4     abatement          act of abating     A47   0.75
2      36d72442aefd8232     abatement         active catalyst     A47   0.25
3      5296b0c19e1ce60e     abatement     eliminating process     A47   0.50
4      54c1e3b9184cb5b6     abatement           forest region     A47   0.00
...                 ...           ...                     ...     ...    ...
36468  8e1386cbefd7f245  wood article          wooden article     B44   1.00
36469  42d9e032d1cd3242  wood article              wooden box     B44   0.50
36470  208654ccb9e14fa3  wood article           wooden handle     B44   0.50
36471  756ec035e694722b  wood article         wooden material     B44   0.75
36472  8d135da0b55b8c88  wood article        wooden substrate     B44   0.50

[36473 rows x 5 columns]>

In [5]:
df_test.head

<bound method NDFrame.head of                   id                       anchor  \
0   4112d61851461f60                     opc drum   
1   09e418c93a776564              adjust gas flow   
2   36baf228038e314b               lower trunnion   
3   1f37ead645e7f0c8                cap component   
4   71a5b6ad068d531f           neural stimulation   
5   474c874d0c07bd21                     dry corn   
6   442c114ed5c4e3c9          tunneling capacitor   
7   b8ae62ea5e1d8bdb      angular contact bearing   
8   faaddaf8fcba8a3f  produce liquid hydrocarbons   
9   ae0262c02566d2ce             diesel fuel tank   
10  a8808e31641e856d            chemical activity   
11  16ae4b99d3601e60         transmit to platform   
12  25c555ca3d5a2092                  oil tankers   
13  5203a36c501f1b7c            generate in layer   
14  b9fdc772bb8fd61c                 slip segment   
15  7aa5908a77a7ec24                   el display   
16  d19ef3979396d47e              overflow device   
17  fd83613b7843

In [6]:
df_submit.head
#print(df_submit.dtypes)
#print('Hello')
df_submit['score']=df_submit['score'].astype('float')
#df_submit = df_submit.astype({'score':'float64'}).dtypes
#print(df_submit.dtypes)

#df.astype({'col1': 'int32'}).dtypes

In [7]:
#data = [['4112d61851461f60',1],['09e418c93a776564',1],['36baf228038e314b',0.25],['1f37ead645e7f0c8',0],['71a5b6ad068d531f',0.5],['474c874d0c07bd21',1],['442c114ed5c4e3c9',0.75],['b8ae62ea5e1d8bdb',0],['faaddaf8fcba8a3f',0.25],['ae0262c02566d2ce',1],['a8808e31641e856d',0.75],['16ae4b99d3601e60',0.25],['25c555ca3d5a2092',1],['5203a36c501f1b7c',1],['b9fdc772bb8fd61c',1],['7aa5908a77a7ec24',0.25],['d19ef3979396d47e',0],['fd83613b7843f5e1',0],['2a619016908bfa45',0.5],['733979d75f59770d',0],['6546846df17f9800',0.75],['3ff0e7a35015be69',0.75],['12ca31f018a2e2b9',0.25],['03ba802ed4029e4d',0],['c404f8b378cbb008',0.75],['78243984c02a72e4',0.25],['de51114bc0faec3e',0.75],['7e3aff857f056bf9',0],['26c3c6dc6174b589',0.25],['b892011ab2e2cabc',1],['8247ff562ca185cc',0.25],['c057aecbba832387',0],['9f2279ce667b21dc',1],['b9ea2b06a878df6f',0.75],['79795133c30ef097',0.25],['25522ee5411e63e9',0]]


In [8]:
def similar(a,b):
    return SequenceMatcher(None, a, b).ratio


In [9]:
df_test['score']= df_test.apply(lambda x: fuzz.partial_ratio(x['anchor'],x['target']),axis=1)
df_test['score'] = df_test['score'].astype('float')
df_test['score'] = df_test['score']/100
df_submit['score'] = df_test['score']
df_submit.head


<bound method NDFrame.head of                   id  score
0   4112d61851461f60   0.75
1   09e418c93a776564   0.67
2   36baf228038e314b   0.64
3   1f37ead645e7f0c8   0.38
4   71a5b6ad068d531f   0.50
5   474c874d0c07bd21   1.00
6   442c114ed5c4e3c9   0.53
7   b8ae62ea5e1d8bdb   0.52
8   faaddaf8fcba8a3f   0.50
9   ae0262c02566d2ce   1.00
10  a8808e31641e856d   0.47
11  16ae4b99d3601e60   0.25
12  25c555ca3d5a2092   0.64
13  5203a36c501f1b7c   0.88
14  b9fdc772bb8fd61c   0.50
15  7aa5908a77a7ec24   0.30
16  d19ef3979396d47e   0.40
17  fd83613b7843f5e1   0.31
18  2a619016908bfa45   0.30
19  733979d75f59770d   0.29
20  6546846df17f9800   0.50
21  3ff0e7a35015be69   0.43
22  12ca31f018a2e2b9   0.67
23  03ba802ed4029e4d   0.20
24  c404f8b378cbb008   0.50
25  78243984c02a72e4   0.62
26  de51114bc0faec3e   0.80
27  7e3aff857f056bf9   0.45
28  26c3c6dc6174b589   0.40
29  b892011ab2e2cabc   0.88
30  8247ff562ca185cc   0.60
31  c057aecbba832387   0.30
32  9f2279ce667b21dc   1.00
33  b9ea2b06a878df

In [10]:
df_submit.to_csv('submission.csv',index=False)
#df_submit.to_csv('/kaggle/working/submission.csv',index=False)

In [11]:
#df_new = pd.read_csv('/kaggle/working/submission.csv')

In [12]:
#df_new.head
